In [1]:
import ipyrad.analysis as ipa
import toyplot
import ipyparallel as ipp

In [2]:
data = "./W_po_17rm.snps.hdf5"

In [3]:
imap = {
#"ref": ["reference"],
"Inam": ["W_po_A815_In", "W_po_T23485_In","W_po_T3645_In","W_po_T4097_In"], #"W_po_A16828_In","W_po_T12362_In",
"Puru": ["W_po_A424_pu","W_po_A7736_pu","W_po_A101_pu","W_po_T12400_pu",  "W_po_T13148_pu", "W_po_A9697_pu","W_po_A10701_pu","W_po_A1381_pu","W_po_A265_pu", "W_po_A2725_pu", "W_po_A2749_pu", "W_po_A2759_pu"],
"JiGu": ["W_po_A308_jigu", "W_po_T7625_jigu", "W_po_T7627_jigu","W_po_T15888_jigu", "W_po_T15889_jigu","W_po_A3264_jigu", "W_po_A334_jigu", "W_po_A368_jigu"],
"Mach": ["W_po_A465_ma","W_po_T2190_ma",  "W_po_T3157_ma",  "W_po_T388_ma",  "W_po_A882_ma", "W_po_J439_ma",  "W_po_A894_ma", "W_po_A896_ma", "W_po_A898_ma", "W_po_A902_ma","W_po_A472_ma","W_po_A24413_ma","W_po_A4235_ma","W_po_J163_ma", "W_po_J192_ma", "W_po_J202_ma", "W_po_J209_ma", "W_po_J222_ma", "W_po_J226_ma", "W_po_J302_ma"], #"W_po_T6424_ma", "W_po_J448_ma", "W_po_J453_ma",
"Roar": ["W_po_J335_roar", "W_po_J340_roar", "W_po_J391_roar", "W_po_J421_roar","W_po_J644_roar", "W_po_J766_roar", "W_po_J768_roar"],
"ArSu": ["W_po_A891_arsu","W_po_80798_arsu", "W_po_81036_arsu", "W_po_85474_arsu", "W_po_86085_arsu","W_po_J561_arsu", "W_po_J563_arsu", "W_po_J610_arsu"],
#"SuTa": ["W_po_A16298_suta","W_po_T11887_suta","W_po_81265_suta", "W_po_T10153_suta", "W_po_T10234_suta"], #"W_po_T14615_suta","W_po_85827_suta","W_po_A12265_suta",
#"Para": [ "W_po_T12793_pa", "W_po_T7132_pa",  "W_po_T9346_pa","W_po_T10694_pa","W_po_25484_pa","W_po_A7346_pa","W_po_A12284_pa",  "W_po_A14569_pa", "W_po_A16227_pa"] #"W_po_A9868_pa",
}
                      
# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [4]:
## look for running ipcluster instance, and create load-balancer
ipyclient = ipp.Client()
print("{} engines found".format(len(ipyclient)))

24 engines found


In [5]:
# init analysis object with input data and (optional) parameter options
struct = ipa.structure(
    name="W_po_west_str",
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.95,
)

Samples: 59
Sites before filtering: 2197855
Filtered (indels): 0
Filtered (bi-allel): 52105
Filtered (mincov): 2053903
Filtered (minmap): 2171590
Filtered (combined): 2172411
Sites after filtering: 25444
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [6]:
struct.mainparams.burnin = 50000
struct.mainparams.numreps = 200000
struct.write_structure_files(abs)

('/array1/lmusher/rio_roosevelt_outfiles/W_po_17rm_outfiles/analysis-structure/tmp-W_po_west_str-<built-in function abs>-1.mainparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/W_po_17rm_outfiles/analysis-structure/tmp-W_po_west_str-<built-in function abs>-1.extraparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/W_po_17rm_outfiles/analysis-structure/tmp-W_po_west_str-<built-in function abs>-1.strfile.txt')

In [7]:
struct.run(nreps=10, kpop=[2, 3, 4, 5, 6, 7, 8], ipyclient=ipyclient, force=True)

Parallel connection | amnh-gen-001.internal.amnh.org: 24 cores
[####################] 100% 5:43:16 | running 70 structure jobs 


In [4]:
struct = ipa.structure(
    data=data, 
    name="W_po_west_str", 
    workdir="analysis-structure",
    imap=imap,
    load_only=True,
)

70 previous results loaded for run [W_po_west_str]


In [5]:
etable = struct.get_evanno_table([2, 3, 4, 5, 6, 7, 8])
etable

,Nreps,deltaK,estLnProbMean,estLnProbStdev,lnPK,lnPPK
2,10,0.000,-19680.81,540.626,0.00,0.00
3,10,0.132,-19490.84,703.170,189.97,93.10
4,10,2.412,-19207.77,916.713,283.07,2210.69
5,10,1.438,-21135.39,5811.234,-1927.62,8355.90
6,10,0.204,-31418.91,15092.167,-10283.52,3073.26
7,10,0.630,-38629.17,21331.296,-7210.26,13429.66
8,10,0.000,-32409.77,13290.046,6219.40,0.00


In [6]:
# get canvas object and set size
canvas = toyplot.Canvas(width=400, height=300)

# plot the mean log probability of the models in red
axes = canvas.cartesian(ylabel="estLnProbMean")
axes.plot(etable.estLnProbMean * -1, color="darkred", marker="o")
axes.y.spine.style = {"stroke": "darkred"}

# plot delta K with its own scale bar of left side and in blue
axes = axes.share("x", ylabel="deltaK", ymax=etable.deltaK.max() + etable.deltaK.max() * .25)
axes.plot(etable.deltaK, color="steelblue", marker="o");
axes.y.spine.style = {"stroke": "steelblue"}

# set x labels
axes.x.ticks.locator = toyplot.locator.Explicit(range(len(etable.index)), etable.index)
axes.x.label.text = "K (N ancestral populations)"

<svg class="toyplot-canvas-Canvas" height="300.0px" id="taf368a9f6835438bb5d0fc7ef266367b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 2 3 4 5 6 7 8 K (N ancestral populations) 20000 30000 40000 estLnProbMean 0 1 2 3 deltaK

In [7]:
k = 2
table = struct.get_clumpp_table(k)

[K2] 10/10 results permuted across replicates (max_var=0).


In [24]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = ["W_po_A815_In", "W_po_T23485_In","W_po_T3645_In","W_po_T4097_In","W_po_A424_pu","W_po_A7736_pu","W_po_A101_pu","W_po_T12400_pu",  "W_po_T13148_pu", "W_po_A9697_pu","W_po_A10701_pu","W_po_A1381_pu","W_po_A265_pu", "W_po_A2725_pu", "W_po_A2749_pu", "W_po_A2759_pu","W_po_A308_jigu", "W_po_T7625_jigu", "W_po_T7627_jigu","W_po_T15888_jigu", "W_po_T15889_jigu","W_po_A3264_jigu", "W_po_A334_jigu", "W_po_A368_jigu","W_po_A465_ma","W_po_T2190_ma",  "W_po_T3157_ma",  "W_po_T388_ma",  "W_po_A882_ma", "W_po_J439_ma",  "W_po_A894_ma", "W_po_A896_ma", "W_po_A898_ma", "W_po_A902_ma","W_po_A472_ma","W_po_A24413_ma","W_po_A4235_ma","W_po_J163_ma", "W_po_J192_ma", "W_po_J202_ma", "W_po_J209_ma", "W_po_J222_ma", "W_po_J226_ma", "W_po_J302_ma","W_po_J335_roar", "W_po_J340_roar", "W_po_J391_roar", "W_po_J421_roar","W_po_J644_roar", "W_po_J766_roar", "W_po_J768_roar","W_po_A891_arsu","W_po_80798_arsu", "W_po_81036_arsu", "W_po_85474_arsu", "W_po_86085_arsu","W_po_J561_arsu", "W_po_J563_arsu", "W_po_J610_arsu"]
table = table.loc[onames]

In [17]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t5cefd5ef2cac42ab8c26d9cb0cefaacc" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> W_po_A815_In W_po_T23485_In W_po_T3645_In W_po_T4097_In W_po_A424_pu W_po_A7736_pu W_po_A101_pu W_po_T12400_pu W_po_T13148_pu W_po_A9697_pu W_po_A10701_pu W_po_A1381_pu W_po_A265_pu W_po_A2725_pu W_po_A2749_pu W_po_A2759_pu W_po_A308_jigu W_po_T7625_jigu W_po_T7627_jigu W_po_T15888_jigu W_po_T15889_jigu W_po_A3264_jigu W_po_A334_jigu W_po_A368_jigu W_po_A465_ma W_po_T2190_ma W_po_T3157_ma W_po_T388_ma W_po_A882_ma W_po_J439_ma W_po_A894_ma W_po_A896_ma W_po_A898_ma W_po_A902_ma W_po_A472_ma W_po_A24413_ma W_po_A4235_ma W_po_J163_ma W_po_J192_ma W_po_J202_ma W_po_J209_ma W_po_J222_ma W_po_J226_ma W_po_J302_ma W_po_J335_roar W_po_J340_roar W_po_J391_roar W_po_J421_roar W_po_J644_roar W_po_J766_roar W_po_J768_roar W_po_A891_arsu W_po_80798_arsu W_po_81036_arsu W_po_85474_arsu W_po_86085_arsu W_po_J561_arsu W_po_J563_arsu W_po_J610_arsu 0.0 0.5 1.0

In [18]:
k = 3
table = struct.get_clumpp_table(k)

# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K3] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="t9c110e59021240229279b00862f7e282" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> W_po_A424_pu W_po_A7736_pu W_po_A101_pu W_po_T12400_pu W_po_T13148_pu W_po_A9697_pu W_po_A10701_pu W_po_A1381_pu W_po_A265_pu W_po_A2725_pu W_po_A2749_pu W_po_A2759_pu W_po_J335_roar W_po_J340_roar W_po_J391_roar W_po_J421_roar W_po_J644_roar W_po_J766_roar W_po_J768_roar W_po_A891_arsu W_po_80798_arsu W_po_81036_arsu W_po_85474_arsu W_po_86085_arsu W_po_J561_arsu W_po_J563_arsu W_po_J610_arsu W_po_A815_In W_po_T23485_In W_po_T3645_In W_po_T4097_In W_po_A308_jigu W_po_T7625_jigu W_po_T7627_jigu W_po_T15888_jigu W_po_T15889_jigu W_po_A3264_jigu W_po_A334_jigu W_po_A368_jigu W_po_A465_ma W_po_T2190_ma W_po_T3157_ma W_po_T388_ma W_po_A882_ma W_po_J439_ma W_po_A894_ma W_po_A896_ma W_po_A898_ma W_po_A902_ma W_po_A472_ma W_po_A24413_ma W_po_A4235_ma W_po_J163_ma W_po_J192_ma W_po_J202_ma W_po_J209_ma W_po_J222_ma W_po_J226_ma W_po_J302_ma 0.0 0.5 1.0

In [25]:
k = 4
table = struct.get_clumpp_table(k)
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = ["W_po_A815_In", "W_po_T23485_In","W_po_T3645_In","W_po_T4097_In","W_po_A424_pu","W_po_A7736_pu","W_po_A101_pu","W_po_T12400_pu",  "W_po_T13148_pu", "W_po_A9697_pu","W_po_A10701_pu","W_po_A1381_pu","W_po_A265_pu", "W_po_A2725_pu", "W_po_A2749_pu", "W_po_A2759_pu","W_po_A308_jigu", "W_po_T7625_jigu", "W_po_T7627_jigu","W_po_T15888_jigu", "W_po_T15889_jigu","W_po_A3264_jigu", "W_po_A334_jigu", "W_po_A368_jigu","W_po_A465_ma","W_po_T2190_ma",  "W_po_T3157_ma",  "W_po_T388_ma",  "W_po_A882_ma", "W_po_J439_ma",  "W_po_A894_ma", "W_po_A896_ma", "W_po_A898_ma", "W_po_A902_ma","W_po_A472_ma","W_po_A24413_ma","W_po_A4235_ma","W_po_J163_ma", "W_po_J192_ma", "W_po_J202_ma", "W_po_J209_ma", "W_po_J222_ma", "W_po_J226_ma", "W_po_J302_ma","W_po_J335_roar", "W_po_J340_roar", "W_po_J391_roar", "W_po_J421_roar","W_po_J644_roar", "W_po_J766_roar", "W_po_J768_roar","W_po_A891_arsu","W_po_80798_arsu", "W_po_81036_arsu", "W_po_85474_arsu", "W_po_86085_arsu","W_po_J561_arsu", "W_po_J563_arsu", "W_po_J610_arsu"]
table = table.loc[onames]

# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K4] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="t9345b532b14a4b22bc7b7981c387b0ab" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> W_po_A815_In W_po_T23485_In W_po_T3645_In W_po_T4097_In W_po_A424_pu W_po_A7736_pu W_po_A101_pu W_po_T12400_pu W_po_T13148_pu W_po_A9697_pu W_po_A10701_pu W_po_A1381_pu W_po_A265_pu W_po_A2725_pu W_po_A2749_pu W_po_A2759_pu W_po_A308_jigu W_po_T7625_jigu W_po_T7627_jigu W_po_T15888_jigu W_po_T15889_jigu W_po_A3264_jigu W_po_A334_jigu W_po_A368_jigu W_po_A465_ma W_po_T2190_ma W_po_T3157_ma W_po_T388_ma W_po_A882_ma W_po_J439_ma W_po_A894_ma W_po_A896_ma W_po_A898_ma W_po_A902_ma W_po_A472_ma W_po_A24413_ma W_po_A4235_ma W_po_J163_ma W_po_J192_ma W_po_J202_ma W_po_J209_ma W_po_J222_ma W_po_J226_ma W_po_J302_ma W_po_J335_roar W_po_J340_roar W_po_J391_roar W_po_J421_roar W_po_J644_roar W_po_J766_roar W_po_J768_roar W_po_A891_arsu W_po_80798_arsu W_po_81036_arsu W_po_85474_arsu W_po_86085_arsu W_po_J561_arsu W_po_J563_arsu W_po_J610_arsu 0.0 0.5 1.0

In [20]:
k = 5
table = struct.get_clumpp_table(k)

# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K5] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="t64b7e28849674301902eb6accac6e56a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> W_po_A424_pu W_po_A7736_pu W_po_A101_pu W_po_T12400_pu W_po_T13148_pu W_po_A9697_pu W_po_A10701_pu W_po_A1381_pu W_po_A265_pu W_po_A2725_pu W_po_A2749_pu W_po_A2759_pu W_po_J335_roar W_po_J340_roar W_po_J391_roar W_po_J421_roar W_po_J644_roar W_po_J766_roar W_po_J768_roar W_po_A891_arsu W_po_80798_arsu W_po_81036_arsu W_po_85474_arsu W_po_86085_arsu W_po_J561_arsu W_po_J563_arsu W_po_J610_arsu W_po_A815_In W_po_T23485_In W_po_T3645_In W_po_T4097_In W_po_A308_jigu W_po_T7625_jigu W_po_T7627_jigu W_po_T15888_jigu W_po_T15889_jigu W_po_A3264_jigu W_po_A334_jigu W_po_A368_jigu W_po_A465_ma W_po_T2190_ma W_po_T3157_ma W_po_T388_ma W_po_A882_ma W_po_J439_ma W_po_A894_ma W_po_A896_ma W_po_A898_ma W_po_A902_ma W_po_A472_ma W_po_A24413_ma W_po_A4235_ma W_po_J163_ma W_po_J192_ma W_po_J202_ma W_po_J209_ma W_po_J222_ma W_po_J226_ma W_po_J302_ma 0.0 0.5 1.0

In [21]:
k = 6
table = struct.get_clumpp_table(k)

# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K6] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="t5e5c512fcb644f23977ba2a6e7a15dc5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> W_po_A424_pu W_po_A7736_pu W_po_A101_pu W_po_T12400_pu W_po_T13148_pu W_po_A9697_pu W_po_A10701_pu W_po_A1381_pu W_po_A265_pu W_po_A2725_pu W_po_A2749_pu W_po_A2759_pu W_po_J335_roar W_po_J340_roar W_po_J391_roar W_po_J421_roar W_po_J644_roar W_po_J766_roar W_po_J768_roar W_po_A891_arsu W_po_80798_arsu W_po_81036_arsu W_po_85474_arsu W_po_86085_arsu W_po_J561_arsu W_po_J563_arsu W_po_J610_arsu W_po_A815_In W_po_T23485_In W_po_T3645_In W_po_T4097_In W_po_A308_jigu W_po_T7625_jigu W_po_T7627_jigu W_po_T15888_jigu W_po_T15889_jigu W_po_A3264_jigu W_po_A334_jigu W_po_A368_jigu W_po_A465_ma W_po_T2190_ma W_po_T3157_ma W_po_T388_ma W_po_A882_ma W_po_J439_ma W_po_A894_ma W_po_A896_ma W_po_A898_ma W_po_A902_ma W_po_A472_ma W_po_A24413_ma W_po_A4235_ma W_po_J163_ma W_po_J192_ma W_po_J202_ma W_po_J209_ma W_po_J222_ma W_po_J226_ma W_po_J302_ma 0.0 0.5 1.0